In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForLanguageModeling, TrainingArguments   
from peft import LoraConfig , get_peft_model
import torch
import transformers

MODEL = "Qwen/Qwen2.5-1.5B"
DATA = "../data/twcs.jsonl"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
ds = torch.load.dataset("json", data_files=DATA, split="train")

def format(example):
    prompt = f"""Instruction: {example['instruction']}
Customer: {example['input']}
Response: """
    output = prompt + example['output'] 
    return {"text": output}

ds = ds.map(format)


model = AutoModelForCausalLM.from_pretrained(
    MODEL,  
    load_in_8bit=True,
    device_map="auto",
)

peft_config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = ["q_proj", "v_proj"],
)

model = get_peft_model(model , peft_config)

args = TrainingArguments(
    output_dir="./lora-ft",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=True,
    logging_steps=20,
    save_steps=500,
    optim="paged_adamw_8bit"
)

collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

model.train()

trainer = transformers.Trainer(
    model=model,
    train_dataset=ds,
    args=args,
    data_collator=collator
)

trainer.train()

model.save_pretrained("./lora-ft")




